In [3]:
import os
import math
import numpy as np
from itertools import combinations

def generate_boundaries(n_shells):
    # Generate increasing boundary sets; the final boundary is fixed at 40 nm.
    if n_shells == 1:
        yield (40,)
    else:
        for comb in combinations(range(36, 40), n_shells - 1):
            yield tuple(list(comb) + [40])

def generate_shell_configurations():
    core_comp = "Pena-Rioux_dielectric_26Au_74Ag"
    shell_comps = [
        "Pena-Rioux_dielectric_34Au_66Ag",
        "Pena-Rioux_dielectric_52Au_48Ag",
        "Pena-Rioux_dielectric_65Au_35Ag",
        "Pena-Rioux_dielectric_76Au_24Ag",
        "Pena-Rioux_dielectric_85Au_15Ag",
        "Pena-Rioux_dielectric_92Au_8Ag",
        "Au_evap"
    ]
    # Outer shell must be gold-rich.
    allowed_gold_rich = {"Pena-Rioux_dielectric_76Au_24Ag",
                         "Pena-Rioux_dielectric_85Au_15Ag",
                         "Pena-Rioux_dielectric_92Au_8Ag",
                         "Au_evap"}
    
    configs = []
    for n_shells in range(1, 6):
        for boundaries in generate_boundaries(n_shells):
            for comps in combinations(shell_comps, n_shells):
                if comps[-1] not in allowed_gold_rich:
                    continue
                configs.append({
                    "n_shells": n_shells,
                    "boundaries": boundaries,
                    "compositions": comps,
                    "core": core_comp
                })
    return configs

def generate_shape_dat(layers):
    dx = dy = dz = 0.004
    L = 0.160
    Nx = Ny = Nz = int(math.ceil(L / dx)) + 1
    lines = [
        ">TARGSPHER  spherical target with explicit spacing",
        f"{Nx * Ny * Nz} = NAT",
        "1.000000  0.000000  0.000000 = A_1 vector",
        "0.000000  1.000000  0.000000 = A_2 vector",
        "1.000000  1.000000  1.000000 = (d_x,d_y,d_z)/d",
        f"-{Nx//2}.5 -{Ny//2}.5 -{Nz//2}.5 = lattice offset",
        "    JA   IX   IY   IZ ICOMP(x,y,z)"
    ]
    JA = 1
    for ix in range(Nx):
        for iy in range(Ny):
            for iz in range(Nz):
                x = (ix - (Nx - 1) / 2) * dx
                y = (iy - (Ny - 1) / 2) * dy
                z = (iz - (Nz - 1) / 2) * dz
                r = np.sqrt(x**2 + y**2 + z**2) * 1e3  # in nm
                comp_idx = 0
                for i, (r_bound, _) in enumerate(layers, start=1):
                    if r <= r_bound:
                        comp_idx = i
                        break
                if comp_idx:
                    lines.append(f"{JA:6d} {ix+1:3d} {iy+1:3d} {iz+1:3d} {comp_idx:1d} {comp_idx:1d} {comp_idx:1d}")
                    JA += 1
    lines[1] = f"{JA - 1} = NAT"
    return "\n".join(lines)

def generate_ddscat_par(layers):
    param_data = [
        "' ========== Parameter file for v7.3 ==================='",
        "'**** Preliminaries ****'",
        "'NOTORQ' = CMTORQ*6 (DOTORQ, NOTORQ) -- either do or skip torque calculations",
        "'PBCGS2' = CMDSOL*6 (PBCGS2, CCG method)",
        "'GPFAFT' = CMETHD*6 (GPFAFT, FFTMKL) -- FFT method",
        "'GKDLDR' = CALPHA*6 (GKDLDR, LATTDR, FLTRCD) -- DDA method",
        "'NOTBIN' = CBINFLAG (NOTBIN, ORIBIN, ALLBIN)",
        "'**** Initial Memory Allocation ****'",
        "100 100 100 = dimensioning allowance for target generation",
        "'**** Target Geometry and Composition ****'",
        "'FROM_FILE' = CSHAPE*9 shape directive",
        "no SHPAR parameters needed",
        f"{len(layers)}         = NCOMP = number of dielectric materials",
    ]
    for i, (_, comp) in enumerate(layers, start=1):
        param_data.append(f"'../../diel/{comp}' = file with refractive index {i}")
    param_data.extend([
        "'**** Additional Nearfield calculation? ****'",
        "0 = NRFLD (=0 to skip nearfield calc., =1 to calculate nearfield E)",
        "0.0 0.0 0.0 0.0 0.0 0.0 (fract. extens. of calc. vol. in -x,+x,-y,+y,-z,+z)",
        "'**** Error Tolerance ****'",
        "1.00e-4 = TOL = MAX ALLOWED (NORM OF |G>=AC|E>-ACA|X>)/(NORM OF AC|E>)",
        "'**** Maximum number of iterations ****'",
        "10000     = MXITER",
        "'**** Integration cutoff parameter for PBC calculations ****'",
        "1.00e-2 = GAMMA (1e-2 is normal, 3e-3 for greater accuracy)",
        "'**** Angular resolution for calculation of <cos>, etc. ****'",
        "0.5\t= ETASCA (number of angles is proportional to [(3+x)/ETASCA]^2 )",
        "'**** Vacuum wavelengths (micron) ****'",
        "0.300 0.600 150 'INV' = wavelengths (first,last,how many,how=LIN,INV,LOG)",
        "'**** Refractive index of ambient medium'",
        "1.333 = NAMBIENT",
        "'**** Effective Radii (micron) **** '",
        "0.040 0.040 1 'LIN' = eff. radii (first, last,how many,how=LIN,INV,LOG)",
        "'**** Define Incident Polarizations ****'",
        "(0,0) (1.,0.) (0.,0.) = Polarization state e01 (k along x axis)",
        "2 = IORTH  (=1 to do only pol. state e01; =2 to also do orth. pol. state)",
        "'**** Specify which output files to write ****'",
        "0 = IWRKSC (=0 to suppress, =1 to write .sca file for each target orient.",
        "'**** Specify Target Rotations ****'",
        "0.    0.   1  = BETAMI, BETAMX, NBETA  (beta=rotation around a1)",
        "0.    0.   1  = THETMI, THETMX, NTHETA (theta=angle between a1 and k)",
        "0.    0.   1  = PHIMIN, PHIMAX, NPHI (phi=rotation angle of a1 around k)",
        "'**** Specify first IWAV, IRAD, IORI (normally 0 0 0) ****'",
        "0   0   0    = first IWAV, first IRAD, first IORI (0 0 0 to begin fresh)",
        "'**** Select Elements of S_ij Matrix to Print ****'",
        "6\t= NSMELTS = number of elements of S_ij to print (not more than 9)",
        "11 12 21 22 31 41\t= indices ij of elements to print",
        "'**** Specify Scattered Directions ****'",
        "'LFRAME' = CMDFRM (LFRAME, TFRAME for Lab Frame or Target Frame)",
        "1 = NPLANES = number of scattering planes",
        "0.  0. 180. 5 = phi, theta_min, theta_max (deg) for plane A",
        "90. 0. 180. 5 = phi, theta_min, theta_max (deg) for plane B"
    ])
    return "\n".join(param_data)

def main():
    configs = generate_shell_configurations()
    base_dir = "all_shells_40nmAeff_GKDLR"
    os.makedirs(base_dir, exist_ok=True)
    csv_entries = []
    
    for cfg in configs:
        n_shells = cfg["n_shells"]
        boundaries = cfg["boundaries"]
        comps = cfg["compositions"]
        core_comp = cfg["core"]
        
        # Build layers: core fixed at 35 nm and shells per the boundaries.
        layers = [(35, core_comp)]
        for b, comp in zip(boundaries, comps):
            layers.append((b, comp))
        
        boundaries_str = "boundaries" + "-".join(str(b) for b in boundaries)
        comps_str = "comps" + "_".join(comp for comp in comps)
        config_dir = os.path.join(base_dir, "outer_40nm", boundaries_str, comps_str)
        os.makedirs(config_dir, exist_ok=True)
        
        shape_content = generate_shape_dat(layers)
        with open(os.path.join(config_dir, "shape.dat"), "w") as f:
            f.write(shape_content)
        
        # Write ddscat.par specific to this configuration.
        ddscat_par_content = generate_ddscat_par(layers)
        with open(os.path.join(config_dir, "ddscat.par"), "w") as f:
            f.write(ddscat_par_content)
        
        # Copy ddscat executable (assumed in current directory).
        ddscat_exe = os.path.join(config_dir, "ddscat")
        os.system(f"cp ddscat {ddscat_exe}")
        os.system(f"chmod +x {ddscat_exe}")
        
        csv_entries.append(config_dir)
    
    with open(os.path.join(base_dir, "config_paths.csv"), "w") as f:
        f.write("path\n" + "\n".join(csv_entries))

if __name__ == "__main__":
    main()


In [ ]:
import os
import math
import numpy as np

def generate_shape_dat(layers):
    dx = dy = dz = 0.004
    L = 0.160
    Nx = Ny = Nz = int(math.ceil(L / dx)) + 1
    lines = [
        ">TARGSPHER  spherical target with explicit spacing",
        f"{Nx * Ny * Nz} = NAT",
        "1.000000  0.000000  0.000000 = A_1 vector",
        "0.000000  1.000000  0.000000 = A_2 vector",
        "1.000000  1.000000  1.000000 = (d_x,d_y,d_z)/d",
        f"-{Nx//2}.5 -{Ny//2}.5 -{Nz//2}.5 = lattice offset",
        "    JA   IX   IY   IZ ICOMP(x,y,z)"
    ]
    JA = 1
    for ix in range(Nx):
        for iy in range(Ny):
            for iz in range(Nz):
                x = (ix - (Nx - 1) / 2) * dx
                y = (iy - (Ny - 1) / 2) * dy
                z = (iz - (Nz - 1) / 2) * dz
                r = np.sqrt(x**2 + y**2 + z**2) * 1e3  # nm
                comp_idx = 0
                for i, (r_bound, _) in enumerate(layers, start=1):
                    if r <= r_bound:
                        comp_idx = i
                        break
                if comp_idx:
                    lines.append(f"{JA:6d} {ix+1:3d} {iy+1:3d} {iz+1:3d} {comp_idx:1d} {comp_idx:1d} {comp_idx:1d}")
                    JA += 1
    lines[1] = f"{JA - 1} = NAT"
    return "\n".join(lines)

def generate_ddscat_par(layers):
    param_data = [
        "' ========== Parameter file for v7.3 ==================='",
        "'**** Preliminaries ****'",
        "'NOTORQ' = CMTORQ*6 (DOTORQ, NOTORQ)",
        "'PBCGS2' = CMDSOL*6 (PBCGS2, CCG method)",
        "'GPFAFT' = CMETHD*6 (GPFAFT, FFTMKL)",
        "'FLTRCD' = CALPHA*6 (GKDLDR, LATTDR, FLTRCD)",
        "'NOTBIN' = CBINFLAG (NOTBIN, ORIBIN, ALLBIN)",
        "'**** Initial Memory Allocation ****'",
        "100 100 100 = dimensioning allowance for target generation",
        "'**** Target Geometry and Composition ****'",
        "'FROM_FILE' = CSHAPE*9 shape directive",
        "no SHPAR parameters needed",
        f"{len(layers)}         = NCOMP = number of dielectric materials",
    ]
    for i, (_, comp) in enumerate(layers, start=1):
        param_data.append(f"'../../../diel/{comp}' = file with refractive index {i}")
    param_data.extend([
        "'**** Additional Nearfield calculation? ****'",
        "0 = NRFLD",
        "0.0 0.0 0.0 0.0 0.0 0.0",
        "'**** Error Tolerance ****'",
        "1.00e-4 = TOL",
        "'**** Maximum number of iterations ****'",
        "10000     = MXITER",
        "'**** Integration cutoff parameter for PBC calculations ****'",
        "1.00e-2 = GAMMA",
        "'**** Angular resolution for calculation of <cos>, etc. ****'",
        "0.5\t= ETASCA",
        "'**** Vacuum wavelengths (micron) ****'",
        "0.300 0.600 150 'INV' = wavelengths",
        "'**** Refractive index of ambient medium'",
        "1.333 = NAMBIENT",
        "'**** Effective Radii (micron) **** '",
        "0.040 0.040 1 'LIN' = eff. radii",
        "'**** Define Incident Polarizations ****'",
        "(0,0) (1.,0.) (0.,0.) = Polarization state e01",
        "2 = IORTH",
        "'**** Specify which output files to write ****'",
        "0 = IWRKSC",
        "'**** Specify Target Rotations ****'",
        "0.    0.   1  = BETAMI, BETAMX, NBETA",
        "0.    0.   1  = THETMI, THETMX, NTHETA",
        "0.    0.   1  = PHIMIN, PHIMAX, NPHI",
        "'**** Specify first IWAV, IRAD, IORI ****'",
        "0   0   0",
        "'**** Select Elements of S_ij Matrix to Print ****'",
        "6\t= NSMELTS",
        "11 12 21 22 31 41\t= indices ij of elements to print",
        "'**** Specify Scattered Directions ****'",
        "'LFRAME' = CMDFRM",
        "1 = NPLANES",
        "0.  0. 180. 5 = phi, theta_min, theta_max for plane A",
        "90. 0. 180. 5 = phi, theta_min, theta_max for plane B"
    ])
    return "\n".join(param_data)

def main():
    # List of available dielectric materials.
    dielectric_list = [
        "Pena-Rioux_dielectric_25Au_75Ag",
        "Pena-Rioux_dielectric_34Au_66Ag",
        "Pena-Rioux_dielectric_52Au_48Ag",
        "Pena-Rioux_dielectric_65Au_35Ag",
        "Pena-Rioux_dielectric_76Au_24Ag",
        "Pena-Rioux_dielectric_85Au_15Ag",
        "Pena-Rioux_dielectric_92Au_8Ag",
        "Au_evap",
        "Ag_evap",
        "25Au75Ag_Rioux_dielec",
        "50Au50Ag_Rioux_dielec",
        "75Au25Ag_Rioux_dielec",

    ]
    # Diameters from 20nm to 40nm in 2nm steps.
    diameters = range(20, 41, 2)
    
    base_dir = "one_component_diameter"
    os.makedirs(base_dir, exist_ok=True)
    csv_entries = []
    
    for diel in dielectric_list:
        for d in diameters:
            # The simulation is for a homogeneous sphere of diameter d.
            radius = d / 2.0  # nm
            layers = [(radius, diel)]
            
            # Directory structure: base_dir/material_<dielectric>/diameter_<d>nm
            config_dir = os.path.join(base_dir, f"material_{diel}", f"diameter_{d}nm")
            os.makedirs(config_dir, exist_ok=True)
            
            # Generate and write shape.dat.
            shape_content = generate_shape_dat(layers)
            with open(os.path.join(config_dir, "shape.dat"), "w") as f:
                f.write(shape_content)
            
            # Generate and write ddscat.par.
            ddscat_par_content = generate_ddscat_par(layers)
            with open(os.path.join(config_dir, "ddscat.par"), "w") as f:
                f.write(ddscat_par_content)
            
            # Copy ddscat executable (assumed present in current directory).
            ddscat_exe = os.path.join(config_dir, "ddscat")
            os.system(f"cp ddscat {ddscat_exe}")
            os.system(f"chmod +x {ddscat_exe}")
            
            csv_entries.append(config_dir)
    
    with open(os.path.join(base_dir, "config_paths.csv"), "w") as f:
        f.write("path\n" + "\n".join(csv_entries))

if __name__ == "__main__":
    main()


In [ ]:
import os
import math
import numpy as np

def generate_shape_dat(layers):
    dx = dy = dz = 0.004
    L = 0.160
    Nx = Ny = Nz = int(math.ceil(L / dx)) + 1
    lines = [
        ">TARGSPHER  spherical target with explicit spacing",
        f"{Nx * Ny * Nz} = NAT",
        "1.000000  0.000000  0.000000 = A_1 vector",
        "0.000000  1.000000  0.000000 = A_2 vector",
        "1.000000  1.000000  1.000000 = (d_x,d_y,d_z)/d",
        f"-{Nx//2}.5 -{Ny//2}.5 -{Nz//2}.5 = lattice offset",
        "    JA   IX   IY   IZ ICOMP(x,y,z)"
    ]
    JA = 1
    for ix in range(Nx):
        for iy in range(Ny):
            for iz in range(Nz):
                x = (ix - (Nx - 1) / 2) * dx
                y = (iy - (Ny - 1) / 2) * dy
                z = (iz - (Nz - 1) / 2) * dz
                r = np.sqrt(x**2 + y**2 + z**2) * 1e3  # in nm
                comp_idx = 0
                for i, (r_bound, _) in enumerate(layers, start=1):
                    if r <= r_bound:
                        comp_idx = i
                        break
                if comp_idx:
                    lines.append(f"{JA:6d} {ix+1:3d} {iy+1:3d} {iz+1:3d} {comp_idx:1d} {comp_idx:1d} {comp_idx:1d}")
                    JA += 1
    lines[1] = f"{JA - 1} = NAT"
    return "\n".join(lines)

def generate_ddscat_par(layers):
    # For a homogeneous sphere, effective radius (in micron) equals physical radius in nm / 1000.
    eff_radius = layers[0][0] / 1000.0  
    param_data = [
        "' ========== Parameter file for v7.3 ==================='",
        "'**** Preliminaries ****'",
        "'NOTORQ' = CMTORQ*6 (DOTORQ, NOTORQ)",
        "'PBCGS2' = CMDSOL*6 (PBCGS2, CCG method)",
        "'GPFAFT' = CMETHD*6 (GPFAFT, FFTMKL)",
        "'GKDLDR' = CALPHA*6 (GKDLDR, LATTDR, FLTRCD)",
        "'NOTBIN' = CBINFLAG (NOTBIN, ORIBIN, ALLBIN)",
        "'**** Initial Memory Allocation ****'",
        "100 100 100 = dimensioning allowance for target generation",
        "'**** Target Geometry and Composition ****'",
        "'FROM_FILE' = CSHAPE*9 shape directive",
        "no SHPAR parameters needed",
        f"{len(layers)}         = NCOMP = number of dielectric materials",
    ]
    for i, (_, comp) in enumerate(layers, start=1):
        param_data.append(f"'../../diel/{comp}' = file with refractive index {i}")
    param_data.extend([
        "'**** Additional Nearfield calculation? ****'",
        "0 = NRFLD",
        "0.0 0.0 0.0 0.0 0.0 0.0",
        "'**** Error Tolerance ****'",
        "1.00e-4 = TOL",
        "'**** Maximum number of iterations ****'",
        "10000     = MXITER",
        "'**** Integration cutoff parameter for PBC calculations ****'",
        "1.00e-2 = GAMMA",
        "'**** Angular resolution for calculation of <cos>, etc. ****'",
        "0.5\t= ETASCA",
        "'**** Vacuum wavelengths (micron) ****'",
        "0.300 0.600 150 'INV' = wavelengths",
        "'**** Refractive index of ambient medium'",
        "1.333 = NAMBIENT",
        "'**** Effective Radii (micron) **** '",
        f"{eff_radius:.3f} {eff_radius:.3f} 1 'LIN' = eff. radii",
        "'**** Define Incident Polarizations ****'",
        "(0,0) (1.,0.) (0.,0.) = Polarization state e01",
        "2 = IORTH",
        "'**** Specify which output files to write ****'",
        "0 = IWRKSC",
        "'**** Specify Target Rotations ****'",
        "0.    0.   1  = BETAMI, BETAMX, NBETA",
        "0.    0.   1  = THETMI, THETMX, NTHETA",
        "0.    0.   1  = PHIMIN, PHIMAX, NPHI",
        "'**** Specify first IWAV, IRAD, IORI ****'",
        "0   0   0",
        "'**** Select Elements of S_ij Matrix to Print ****'",
        "6\t= NSMELTS",
        "11 12 21 22 31 41\t= indices ij of elements to print",
        "'**** Specify Scattered Directions ****'",
        "'LFRAME' = CMDFRM",
        "1 = NPLANES",
        "0.  0. 180. 5 = phi, theta_min, theta_max for plane A",
        "90. 0. 180. 5 = phi, theta_min, theta_max for plane B"
    ])
    return "\n".join(param_data)

def main():
    # List of available dielectric materials.
    dielectric_list = [
        "Pena-Rioux_dielectric_26Au_74Ag",
        "Pena-Rioux_dielectric_34Au_66Ag",
        "Pena-Rioux_dielectric_52Au_48Ag",
        "Pena-Rioux_dielectric_65Au_35Ag",
        "Pena-Rioux_dielectric_76Au_24Ag",
        "Pena-Rioux_dielectric_85Au_15Ag",
        "Pena-Rioux_dielectric_92Au_8Ag",
        "Au_evap",
        "Ag_evap",
        "25Au75Ag_Rioux_dielec",
        "50Au50Ag_Rioux_dielec",
        "75Au25Ag_Rioux_dielec",

    ]
    # Diameters from 20nm to 40nm in 2nm steps.
    diameters = range(10, 81, 2)
    
    base_dir = "one_component_diameter"
    os.makedirs(base_dir, exist_ok=True)
    csv_entries = []
    
    for diel in dielectric_list:
        for d in diameters:
            radius = d / 2.0  # in nm
            layers = [(radius, diel)]
            
            config_dir = os.path.join(base_dir, f"material_{diel}", f"diameter_{d}nm")
            os.makedirs(config_dir, exist_ok=True)
            
            shape_content = generate_shape_dat(layers)
            with open(os.path.join(config_dir, "shape.dat"), "w") as f:
                f.write(shape_content)
            
            ddscat_par_content = generate_ddscat_par(layers)
            with open(os.path.join(config_dir, "ddscat.par"), "w") as f:
                f.write(ddscat_par_content)
            
            ddscat_exe = os.path.join(config_dir, "ddscat")
            os.system(f"cp ddscat {ddscat_exe}")
            os.system(f"chmod +x {ddscat_exe}")
            
            csv_entries.append(config_dir)
    
    with open(os.path.join(base_dir, "config_paths.csv"), "w") as f:
        f.write("path\n" + "\n".join(csv_entries))

if __name__ == "__main__":
    main()
